In [ ]:
import pickle
with open("./Datasets/dataset.pkl", 'rb') as file:
    all_data = pickle.load(file)

In [ ]:
import torch

def formatModelData(all_data, paths):
    all_inputs = []
    all_outputs = []

    for path in paths:
        all_inputs.extend(all_data[path]['inputs'])
        all_outputs.extend(all_data[path]['outputs'])
    
    num_classes = 3
    
    inputs_tensor = torch.tensor(all_inputs, dtype=torch.float32)
    
    # Conver outputs to one-hot vectors 
    oneHotOutputs = []
    for output in all_outputs:
        one_hot = torch.zeros(num_classes, dtype=torch.float32)
        one_hot[output] = 1.0
        oneHotOutputs.append(one_hot)
    
    outputs_tensor = torch.stack(oneHotOutputs)
    
    return inputs_tensor, outputs_tensor

In [ ]:
test_name = './videos/IMG_2478.mov'
test_input, test_output = formatModelData(all_data, [test_name])

train_names = []
for key in all_data.keys():
    if key != test_name:
        train_names.append(key)

X, y = formatModelData(all_data, train_names)

In [ ]:
import copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
 

X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.7, shuffle=True)
 
class Multiclass(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(38, 30)  
        self.act1 = nn.ReLU()             
        self.hidden2 = nn.Linear(30, 20)  
        self.act2 = nn.ReLU()             
        self.output = nn.Linear(20, 3)
 
    def forward(self, x):
        x = self.act1(self.hidden1(x))    
        x = self.act2(self.hidden2(x))    
        x = self.output(x)       
        return x
 
 
model = Multiclass()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
 
n_epochs = 40
batch_size = 4
batches_per_epoch = len(X_train) // batch_size
 
best_acc = - np.inf   # init to negative infinity
best_weights = None
train_loss_hist = []
train_acc_hist = []
val_loss_hist = []
val_acc_hist = []
 
# training loop
for epoch in range(n_epochs):
    epoch_loss = []
    epoch_acc = []
    # set model in training mode and run through each batch
    model.train()
    with tqdm.trange(batches_per_epoch, unit="batch", mininterval=0) as bar:
        bar.set_description(f"Epoch {epoch}")
        for i in bar:
            start = i * batch_size
            X_batch = X_train[start:start+batch_size]
            y_batch = y_train[start:start+batch_size]
            # forward pass
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch)
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            # update weights
            optimizer.step()
            # compute and store metrics
            acc = (torch.argmax(y_pred, 1) == torch.argmax(y_batch, 1)).float().mean()
            epoch_loss.append(float(loss))
            epoch_acc.append(float(acc))
            bar.set_postfix(
                loss=float(loss),
                acc=float(acc)
            )
    # set model in evaluation mode and run through the test set
    model.eval()
    y_pred = model(X_val)
    ce = loss_fn(y_pred, y_val)
    acc = (torch.argmax(y_pred, 1) == torch.argmax(y_val, 1)).float().mean()
    ce = float(ce)
    acc = float(acc)
    train_loss_hist.append(np.mean(epoch_loss))
    train_acc_hist.append(np.mean(epoch_acc))
    val_loss_hist.append(ce)
    val_acc_hist.append(acc)
    if acc > best_acc:
        best_acc = acc
        best_weights = copy.deepcopy(model.state_dict())
    print(f"Epoch {epoch} validation: Cross-entropy={ce:.2f}, Accuracy={acc*100:.1f}%")
 
    # Restore best model
model.load_state_dict(best_weights)
 
# Plot the loss and accuracy
plt.plot(train_loss_hist, label="train")
plt.plot(val_loss_hist, label="val")
plt.xlabel("epochs")
plt.ylabel("cross entropy")
plt.legend()
plt.show()
 
plt.plot(train_acc_hist, label="train")
plt.plot(val_acc_hist, label="val")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.legend()
plt.show()

In [ ]:
plt.plot(train_acc_hist, label="train")
plt.plot(val_acc_hist, label="val")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.legend()
plt.show()

In [ ]:
#Separate test set

model.eval()

with torch.no_grad():
    y_pred_test = model(test_input)
    test_loss = loss_fn(y_pred_test, test_output)
    test_accuracy = (torch.argmax(y_pred_test, 1) == torch.argmax(test_output, 1)).float().mean()

print(f"Test Loss: {test_loss.item():.4f}")
print(f"Test Accuracy: {test_accuracy.item()*100:.2f}%")

In [ ]:
save_path = 'model_best_weights.pth'
torch.save(best_weights, save_path)

In [ ]:
model.eval()

with torch.no_grad():
    y_pred = model(X_val)
    train_loss = loss_fn(y_pred, y_val)
    train_accuracy = (torch.argmax(y_pred, 1) == torch.argmax(y_val, 1)).float().mean()

print(f"Train Loss: {train_loss.item():.4f}")
print(f"Train Accuracy: {train_accuracy.item()*100:.2f}%")

In [ ]:
model.eval()

with torch.no_grad():
    y_pred = model(X_train)
    train_loss = loss_fn(y_pred, y_train)
    train_accuracy = (torch.argmax(y_pred, 1) == torch.argmax(y_train, 1)).float().mean()

print(f"Train Loss: {train_loss.item():.4f}")
print(f"Train Accuracy: {train_accuracy.item()*100:.2f}%")